In [1]:
! export CUDA_VISIBLE_DEVICES=""

In [2]:
import torch
# torch.cuda.empty_cache()
torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pathlib

import numpy as np
import pandas as pd

from sops_anomaly.datasets import MNIST, SupervisedDataset
from sops_anomaly.detectors import AutoEncoder
from sops_anomaly.evaluation import Result

In [9]:
RESULTS_ROOT = pathlib.Path("../results")
RESULT_FILE = "autoencoder.csv"
N_TRAIN_SAMPLES = 1000
N_TEST_SAMPLES = 100

if not RESULTS_ROOT.exists():
    RESULTS_ROOT.mkdir()

In [5]:
def best_result(predictions, targets, max_error):
    best_f1, bets_threshold = 0, 0
    best_result = None
    for threshold in range(10):
        labels = (predictions > max_error*threshold*0.1).astype(np.int32)
        result = Result(labels, targets)
        if result.f1 > best_f1:
            best_f1 = result.f1
            best_result = result
            best_threshold = threshold * 0.1
    return best_result

In [6]:
results = {}
for anomaly_class in range(10):
    mnist = MNIST(anomaly_class=anomaly_class)
    mnist = SupervisedDataset(mnist)
    train_data = mnist.get_train_samples(n_samples=N_TRAIN_SAMPLES)
    test_data, test_targets = mnist.get_test_samples(n_samples=N_TEST_SAMPLES)

    model = AutoEncoder(window_size=1, latent_size=50, layers=(200,))
    model.train(train_data, epochs=5)
    predictions = model.predict(test_data)
    
    results[anomaly_class] = best_result(predictions, test_targets, model.max_error)
    
results['total'] = sum(list(results.values()), Result([],[]))

In [7]:
index = list(range(10)) + ['total']
columns = ['accuracy', 'f1_score', 'precision', 'recall']
scores = []
for i in index:
    result = results[i]
    scores.append([result.accuracy, result.f1, result.precision, result.recall])
    
pd.DataFrame(data=scores, index=index, columns=columns).to_csv((RESULTS_ROOT / RESULT_FILE))

In [8]:
# given: model, train_size, test_size
# for anomaly_class in range(10)
# data = MNIST(anomaly_class)
# model.train(data.train_data)
# predictions = model.predict(data.test_data)
# score[anomaly_class] = best_result(predictions, data.test_targets, model.max_error)
# build dataframe index(0...9,'all'), colums=[f1, recall, precision, etc]
# save results to csv 
# be able to run it for any model 